# Day 3: Disney Parks Guest Spending Behavior

You are a data analyst working with the Disney Parks revenue team to understand nuanced guest spending patterns across different park experiences. The team wants to develop a comprehensive view of visitor purchasing behaviors. Your goal is to uncover meaningful insights that can drive personalized marketing strategies.

In [ ]:
import pandas as pd
import numpy as np

fct_guest_spending_data = [
  {
    "guest_id": 1,
    "visit_date": "2024-07-05",
    "amount_spent": 50,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-07-06",
    "amount_spent": 30,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-07-10",
    "amount_spent": 20.5,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 4,
    "visit_date": "2024-07-12",
    "amount_spent": 40,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-07-15",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 5,
    "visit_date": "2024-07-20",
    "amount_spent": 60,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 6,
    "visit_date": "2024-07-25",
    "amount_spent": 25,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-03",
    "amount_spent": 55,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-15",
    "amount_spent": 45,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-05",
    "amount_spent": 22,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-20",
    "amount_spent": 38,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 7,
    "visit_date": "2024-08-10",
    "amount_spent": 15,
    "park_experience_type": "Character Meet"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-25",
    "amount_spent": 28,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-27",
    "amount_spent": 32,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-02",
    "amount_spent": 65,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-05",
    "amount_spent": 50,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 9,
    "visit_date": "2024-09-15",
    "amount_spent": 40,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 10,
    "visit_date": "2024-09-20",
    "amount_spent": 70,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-25",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-28",
    "amount_spent": 10,
    "park_experience_type": "Character Meet"
  }
]
fct_guest_spending = pd.DataFrame(fct_guest_spending_data)


## Question 1

What is the average spending per guest per visit for each park experience type during July 2024? Ensure that park experience types with no recorded transactions are shown with an average spending of 0.0. This analysis helps establish baseline spending differences essential for later segmentation.

In [ ]:
filtro = (fct_guest_spending['visit_date']>='2024-07-01')&(fct_guest_spending['visit_date']<='2024-07-31')
fct_guest_spending_october= fct_guest_spending.loc[filtro]
avg_total_expending = fct_guest_spending_october.groupby(['guest_id','park_experience_type'])['amount_spent'].mean()
avg_total_expending = avg_total_expending.reset_index()
park_experience_type = fct_guest_spending['park_experience_type'].unique()
park_experience_type = pd.DataFrame({'park_experience_type':park_experience_type})
park_experience_type = park_experience_type.reset_index()
merge = park_experience_type.merge(avg_total_expending,on="park_experience_type",how="left")
merge = merge.fillna(0.0)
merge = merge.groupby('park_experience_type')['amount_spent'].mean().reset_index()
merge= merge.rename(columns={'amount_spent':'avg_spending_per_guest'})
merge

## Question 2

For guests who visited our parks more than once in August 2024, what is the difference in spending between their first and their last visit? This investigation, using sequential analysis, will reveal any shifts in guest spending behavior over multiple visits.

In [ ]:
fct_guest_spending['visit_date'] = pd.to_datetime(fct_guest_spending['visit_date'],errors='coerce')
filtro = (fct_guest_spending['visit_date']>='2024-08-01')&(fct_guest_spending['visit_date']<='2024-08-31')
fct_august= fct_guest_spending.loc[filtro]
guest_counts = fct_august['guest_id'].value_counts()
mult_guest = guest_counts[guest_counts>=2].index
fct_august_multiple = fct_august[fct_august['guest_id'].isin(mult_guest)]
#fct_august_multiple = fct_august_multiple.groupby(['guest_id','visit_date'])['amount_spent']
fct_august_multiple = fct_august_multiple.sort_values(by=['guest_id','visit_date'])
fct_august_multiple = fct_august_multiple.groupby('guest_id')['amount_spent']
first_spend = fct_august_multiple.first()
last_spend = fct_august_multiple.last()
spending_diff = (last_spend - first_spend).reset_index(name='spending_diff')
print(spending_diff)

## Question 3

In September 2024, how can guests be categorized into distinct spending segments such as Low, Medium, and High based on their total spending? Use the following thresholds for categorization: 
-Low: Includes values from $0 up to, but not including, $50.
-Medium: Includes values from $50 up to, but not including, $100.
-High: Includes values from $100 and above. 
Exclude guests who did not make any purchases in the period.

In [ ]:
filtro = (fct_guest_spending['visit_date']>='2024-09-01')&(fct_guest_spending['visit_date']<='2024-09-30')
visitas_septiembre = fct_guest_spending.loc[filtro]
visitas_septiembre = visitas_septiembre.groupby('guest_id')['amount_spent'].sum().reset_index()
visitas_septiembre['spending_segment'] = pd.cut(visitas_septiembre['amount_spent'],bins=[0,50,100,float('inf')],labels=['Low','Medium','High'],right=False)
visitas_septiembre = visitas_septiembre[visitas_septiembre['amount_spent']!=0]
visitas_septiembre

Made with ❤️ by [Interview Master](https://www.interviewmaster.ai)